In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import geopandas

import warnings

warnings.filterwarnings('ignore')

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
file_name = f'./dbs/intermediate/stops_1_new.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)
# df = df[df.holiday_s != 1]
# df = df[df.weekday_s == 1]
# df = df.drop(['holiday_s', 'weekday_s'], axis=1)
df.drop_duplicates(subset=['uid', 'cluster'], keep='first', inplace=True)

In [4]:
# read home clusters

df_h = pd.read_parquet(f'./dbs/intermediate/home_inference.parquet')
df_h.drop_duplicates(subset='uid', keep='first', inplace=True)
df_h['home_potential'] = 1
df_h['work_potential'] = 0

In [5]:
# read work clusters, keep only ppl having home locations

df_w = pd.read_parquet(f'./dbs/intermediate/work_inference.parquet')
df_w.drop_duplicates(subset='uid', keep='first', inplace=True)
df_w['home_potential'] = 0

ppl_having_home = df_h.uid.unique()
df_w = df_w[df_w.uid.isin(ppl_having_home)]

In [6]:
# concat home and work clusters and create activity table

df_act = pd.concat([df_h[['uid', 'cluster', 'home_potential', 'work_potential']], df_w[['uid', 'cluster', 'home_potential', 'work_potential']]],axis=0, ignore_index=True).sort_values(by='uid')

In [7]:
# add lat and long information of the clusters

df_act = pd.merge(df_act, df[['uid', 'cluster', 'cluster_lat', 'cluster_lng' ]], on=['uid', 'cluster'], how="left")


In [8]:
# read population by deso zones

DeSO = geopandas.read_file(
    f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Alder_region.shp')

DeSO.loc[:, 'adult_pop'] = DeSO.loc[:, [ 'Ald20_24', 'Ald25_29', 'Ald30_34', 'Ald35_39', 'Ald40_44', 'Ald45_49', 'Ald50_54', 'Ald55_59', 'Ald60_64', 'Ald65_69', 'Ald70_74', 'Ald75_79', 'Ald80_w']].sum(axis=1) + DeSO.loc[:,'Ald16_19']/2


print(DeSO.crs)

DeSO.to_crs(4326, inplace=True)
print(DeSO.crs)

PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3006"]]
epsg:4326


In [9]:
# read employee numbers by deso zones
# multiply the number of employees by work activity participation ratio

DeSO_arb = geopandas.read_file(
    f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Sysselsattning_region.shp')

DeSO_arb['county'] = DeSO_arb['Deso'].str[:1]
DeSO_arb['county'] = DeSO_arb['Deso'].str[:2].astype(int)

work_participation = pd.read_csv('dbs/intermediate/work_participation.csv',index_col=0)

DeSO_arb = pd.merge(DeSO_arb, work_participation, left_on=['county'], right_on=['home_county'], how="left")



DeSO_arb['Forvarb'] = DeSO_arb['Forvarb']*DeSO_arb['nunique']

In [10]:
DeSO = pd.merge(DeSO, DeSO_arb[['Deso','Forvarb']], left_on=['Deso'], right_on=['Deso'], how="left")


DeSO.rename(columns={'Forvarb':'employee_pop'}, inplace=True )

In [11]:
# add deso information to the activity table

gdf_act = geopandas.GeoDataFrame(df_act, geometry=geopandas.points_from_xy(df_act.cluster_lng, df_act.cluster_lat), crs="EPSG:4326")


gdf_act = gdf_act.sjoin(DeSO[['Deso','geometry', 'adult_pop','employee_pop']], how="left")

In [12]:
# count the number of detected homes for each deso zones

gdf_2_home = gdf_act[gdf_act.home_potential== 1]

gdf_2_home= gdf_2_home.groupby('Deso')['uid'].count().reset_index().rename(columns={"uid":'home_count'})

In [13]:
gdf_2_home = pd.merge(DeSO[['Deso','adult_pop', 'employee_pop']], gdf_2_home, left_on=['Deso'], right_on=['Deso'], how="left")

## calculate a weight for each device
- we use inverse probability weighting (IPW) to assign each device a weight, i.e., the reverse of the phone users’ count ratio to the DeSO zone’s actual adult population size (Wp).
- we fit the calculated weights to the work activity participation rates by deso zone, to ensure that the work activity participation rate is accurately captured. E.g., 60% of people participate in work activity by the survey, but we detect only 20% work activity participation in the data.  By the statistic, the weight of the devices with work activity is increased and the weight of the devices with no work activity is decreased.
- we apply weight trimming technique, where any weight above the cut-point weight (W0) is set to W0.

In [14]:
gdf_2_home.loc[:, 'wt_p'] = gdf_2_home.loc[:, 'adult_pop'] / gdf_2_home.loc[:, 'home_count']


In [15]:
#w0 = ((np.std(gdf_2_home.loc[:, 'wt_p']) / np.mean(gdf_2_home.loc[:, 'wt_p'])) ** 2 + 1) ** 0.5 * 3.5 * np.median(   gdf_2_home.loc[:, 'wt_p'])

In [16]:
#gdf_2_home.loc[gdf_2_home['wt_p'] > w0, 'wt_p'] = w0

print('Population above 17 years old: ',np.sum(gdf_2_home['wt_p'] * gdf_2_home['home_count']))

Population above 17 years old:  8060839.0


In [17]:
employees = df_act['uid'][df_act.work_potential== 1].unique()


In [18]:
gdf_act = gdf_act[gdf_act.home_potential == 1]
gdf_act['work_potential'][gdf_act.uid.isin(employees)] = 1

In [19]:
gdf_act = pd.merge(gdf_act, gdf_2_home[['Deso','wt_p']], on='Deso', how='left')

In [20]:
df_act = gdf_act[['uid', 'Deso', 'work_potential', 'adult_pop', 'employee_pop', 'wt_p']].sort_values(by=['Deso'])

In [21]:
def ipf_wtp(data):
    current_pop = data['wt_p'].sum()
    #pop_factor = current_pop/data['adult_pop']
    for it in range(20):

        employees = data['wt_p'][data['work_potential']==1].sum() #* pop_factor
        data['wt_p'][data['work_potential']==1] = data['wt_p'][data['work_potential']==1] * (data['employee_pop']/ employees)
        population = data['wt_p'].sum()
        data['wt_p'] = data['wt_p'] * (current_pop/ population)
    return data

In [22]:
# fit the weights to the work activity participation ratios

tqdm.pandas()
df_act = df_act.groupby('Deso').progress_apply(ipf_wtp)

  0%|          | 0/5985 [00:00<?, ?it/s]

In [23]:
w0 = ((np.std(df_act.loc[:, 'wt_p']) / np.mean(df_act.loc[:, 'wt_p'])) ** 2 + 1) ** 0.5 * 3.5 * np.median( df_act.loc[:, 'wt_p'])
df_act.loc[df_act['wt_p'] > w0, 'wt_p'] = w0
print('sum of all weights', df_act['wt_p'].sum())

In [26]:
# save the table

df_act.to_csv(f'./dbs/intermediate/indi_weights.csv')

## in the script 1.1, use an algorithm looking within the sweden polygone instead of spatial join with deso
## in the script 1.1, directly remove the holidays and save the ready data
## in the script 1.1, save the building locations as lat long as before
## in the script 2.2, keep only people above 18


In [27]:
# def ipf_wtp(data):
#     current_pop = data['wt_p'].sum()
#     #pop_factor = current_pop/data['adult_pop']
#     std_eror =10
#     while std_eror >  0.0001:
#
#         employees = data['wt_p'][data['work_potential']==1].sum() #* pop_factor
#         data['wt_p'][data['work_potential']==1] = data['wt_p'][data['work_potential']==1] * (data['employee_pop']/ employees)
#         population = data['wt_p'].sum()
#         wt_p_old = data['wt_p']
#
#         data['wt_p'] = data['wt_p'] * (current_pop/ population)
#         std_eror = np.sqrt(np.sum(np.square((wt_p_old - data['wt_p'])))/len(data))
#
#
#     return data

In [28]:
# tqdm.pandas()
# df_act1 = df_act.groupby('Deso').progress_apply(ipf_wtp)